# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [183]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))
    
    station_locations_by_hash = df[df['hash'] == hashid]
    #print(station_locations_by_hash.count())
    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [188]:
import pandas as pd
import numpy as np
df=pd.read_csv("data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv")

df.Data_Value *= 0.1 
df.Date = pd.to_datetime(df['Date']) 


df['Day'] = pd.DatetimeIndex(df.Date).day 
df['Month'] = pd.DatetimeIndex(df.Date).month 
df['Year'] = pd.DatetimeIndex(df.Date).year

df = df.set_index(['Month','Day']) 
df.sort_index(inplace = True)

feb_df = df.loc[2,29] 
df = df[~ df.index.isin(feb_df.index)] 
#df.reset_index()
#df.shape

In [189]:
df_14 = df[df.Year != 2015] # years excluding 2015
df_14_max_temp = df_14[df_14.Element == 'TMAX'] 
df_14_min_temp = df_14[df_14.Element == 'TMIN'] 

max_temp_14 = df_14_max_temp.groupby(level = ['Month', 'Day']).Data_Value.max()  
min_temp_14 = df_14_min_temp.groupby(level = ['Month', 'Day']).Data_Value.min()
#min_temp_14.head()

In [190]:
df_15 = df[df.Year == 2015] # only year 2015
df_15_max_temp = df_15[df_15.Element == 'TMAX']
df_15_min_temp = df_15[df_15.Element == 'TMIN'] 

max_temp_15 = df_15_max_temp.groupby(level = ['Month', 'Day'])['Data_Value', 'Date'].max()
min_temp_15 = df_15_min_temp.groupby(level = ['Month', 'Day'])['Data_Value', 'Date'].min()
#min_temp_15.head()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as dt
import matplotlib.ticker as ticker

f, ax = plt.subplots(figsize = (16,7)) 
ax.yaxis.grid() 
#ax.xaxis.grid()
ax.set_title('New Orleans Record Temperatures 2005 - 2014 and 2015')
ax.set_xlabel('Temperature Summary plot near Ann Arbor, Michigan, United States ')
ax.set_ylabel('Temperatures in $(^{\circ}$C)') 

dates = np.unique(df_15['Date'])

plt.plot(dates, max_temp_14, 'red',  linewidth = 0.5 , label = '2005-2014 High Temp')
plt.plot(dates, min_temp_14, 'blue', linewidth = 0.5  , label = '2005-2014 Low Temp') 
plt.fill_between(dates, min_temp_14, max_temp_14, facecolor='#feffce')

max_label='2015 High Temp'
for i, rows in max_temp_15.iterrows():
    if rows.Data_Value > max_temp_14.loc[i]:
        plt.scatter(rows.Date, rows.Data_Value, color = '#300001', marker = '*', label = max_label)
        max_label = '_nolegend_' 

min_label='2015 Low Temp'
for i, rows in min_temp_15.iterrows():
    if rows.Data_Value < min_temp_14.loc[i]:
        plt.scatter(rows.Date, rows.Data_Value, color = '#03026d', marker = '+', label = min_label)
        min_label = '_nolegend_' 

plt.legend()
        
ax.set_xlim(dates[0], dates[-1])
ax.set_ylim(-40,50)

ax.xaxis.set_minor_locator(dt.MonthLocator(bymonthday=15))
ax.xaxis.set_major_formatter(ticker.NullFormatter())
ax.xaxis.set_minor_formatter(dt.DateFormatter('%b'))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(5))

ax2 = ax.twinx() 
ax2.set_ylabel('Temperatures $(^{\circ}$F)') 
def temp_con(c):  
    return 9/5 *c + 32
ax2.set_ylim(temp_con(-10),temp_con(35)) 
ax2.yaxis.set_major_locator(ticker.MultipleLocator(20))
ax2.yaxis.set_minor_locator(ticker.MultipleLocator(10))


plt.show()
